# Mistral Fine-tuning API

Check out the docs: https://docs.mistral.ai/capabilities/finetuning/

In [20]:
!pip install mistralai pandas


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


## Import the dataset

Let's use the medical_knowledge_from_extracts dataset where the desired output format is a JSON object with:

> * conditions
> * interventions, which can be further categorized into behavioral, drugs and other types. An example of output would look like this:

{
    "conditions": "Proteinuria",
    "interventions": "Drug: Losartan Potassium|Other: Comparator: Placebo (Losartan)|Drug: Comparator: amlodipine besylate|Other: Comparator: Placebo (amlodipine besylate)|Other: Placebo (Losartan)|Drug: Enalapril Maleate"
}

In [21]:
import pandas as pd
# Lecture du fichier .jsonl
df = pd.read_json("medical_data_without_instructions.jsonl", lines=True)

df_train=df.sample(frac=0.995,random_state=200)
df_eval=df.drop(df_train.index)

df_train.to_json("medical_data_train.jsonl", orient="records", lines=True)
df_eval.to_json("medical_data_eval.jsonl", orient="records", lines=True)

In [22]:
df_train.iloc[102]['messages']

[{'role': 'user',
  'content': 'Viusid, a nutritional supplement, increases survival and reduces disease progression in HCV-related decompensated cirrhosis: a randomised and controlled trial.  Viusid is a nutritional supplement with recognised antioxidant and immunomodulatory properties which could have beneficial effects on cirrhosis-related clinical outcomes such as survival, disease progression and development of hepatocellular carcinoma (HCC). This study evaluated the efficacy and safety of viusid in patients with HCV-related decompensated cirrhosis. A randomised double-blind and placebo-controlled study was conducted in a tertiary care academic centre (National Institute of Gastroenterology, Havana, Cuba). The authors randomly assigned 100 patients with HCV-related decompensated cirrhosis to receive viusid (three oral sachets daily, n=50) or placebo (n=50) during 96 weeks. The primary outcome of the study was overall survival at 96 weeks, and the secondary outcomes included time t

## Upload dataset

In [23]:
from mistralai import Mistral
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ["MISTRAL_API_KEY"]

client = Mistral(api_key=api_key)

medical_data_train = client.files.upload(file={
    "file_name": "medical_data_train.jsonl",
    "content": open("medical_data_train.jsonl", "rb"),
})
medical_data_eval = client.files.upload(file={
    "file_name": "medical_data_eval.jsonl",
    "content": open("medical_data_eval.jsonl", "rb"),
})

In [24]:
import json
def pprint(obj):
    print(json.dumps(obj.dict(), indent=4))

In [25]:
pprint(medical_data_train)

{
    "id": "c69b7154-1577-43f1-b718-39aa4a9f9f70",
    "object": "file",
    "bytes": null,
    "created_at": 1750344476,
    "filename": "medical_data_train.jsonl",
    "purpose": "fine-tune",
    "sample_type": "instruct",
    "source": "upload",
    "num_lines": 1376
}


/var/folders/bt/j58s9sqj2p534q65czjjc7fh0000gn/T/ipykernel_64546/3084144206.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(json.dumps(obj.dict(), indent=4))


In [26]:
pprint(medical_data_eval)

{
    "id": "8ce51668-728f-47cc-8f9c-0011ffae3ffa",
    "object": "file",
    "bytes": null,
    "created_at": 1750344476,
    "filename": "medical_data_eval.jsonl",
    "purpose": "fine-tune",
    "sample_type": "instruct",
    "source": "upload",
    "num_lines": 7
}


/var/folders/bt/j58s9sqj2p534q65czjjc7fh0000gn/T/ipykernel_64546/3084144206.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(json.dumps(obj.dict(), indent=4))


## Create a fine-tuning job

In [27]:
created_jobs = client.fine_tuning.jobs.create(
    model="ministral-3b-latest",
    training_files=[{"file_id": medical_data_train.id, "weight": 1}],
    validation_files=[medical_data_eval.id],
    hyperparameters={
    "training_steps": 10,
    "learning_rate":0.0001
    },
    auto_start=True
)
created_jobs

CompletionJobOut(id='634c2e1a-6b03-4260-86f7-f9eb3fe9124f', auto_start=True, model='ministral-3b-latest', status='QUEUED', created_at=1750669586, modified_at=1750669586, training_files=['c69b7154-1577-43f1-b718-39aa4a9f9f70'], hyperparameters=CompletionTrainingParameters(training_steps=10, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=None, seq_len=32768, fim_ratio=None), validation_files=['8ce51668-728f-47cc-8f9c-0011ffae3ffa'], object='job', fine_tuned_model=None, suffix=None, integrations=[], trained_tokens=None, metadata=JobMetadataOut(expected_duration_seconds=None, cost=0.0, cost_currency=None, train_tokens_per_step=None, train_tokens=None, data_tokens=None, estimated_start_time=None), job_type='completion', repositories=[])

In [28]:
pprint(created_jobs)

{
    "id": "634c2e1a-6b03-4260-86f7-f9eb3fe9124f",
    "auto_start": true,
    "model": "ministral-3b-latest",
    "status": "QUEUED",
    "created_at": 1750669586,
    "modified_at": 1750669586,
    "training_files": [
        "c69b7154-1577-43f1-b718-39aa4a9f9f70"
    ],
    "hyperparameters": {
        "training_steps": 10,
        "learning_rate": 0.0001,
        "weight_decay": 0.1,
        "warmup_fraction": 0.05,
        "epochs": null,
        "seq_len": 32768,
        "fim_ratio": null
    },
    "validation_files": [
        "8ce51668-728f-47cc-8f9c-0011ffae3ffa"
    ],
    "object": "job",
    "fine_tuned_model": null,
    "suffix": null,
    "integrations": [],
    "trained_tokens": null,
    "metadata": {
        "expected_duration_seconds": null,
        "cost": 0.0,
        "cost_currency": null,
        "train_tokens_per_step": null,
        "train_tokens": null,
        "data_tokens": null,
        "estimated_start_time": null
    },
    "job_type": "completion",
    

/var/folders/bt/j58s9sqj2p534q65czjjc7fh0000gn/T/ipykernel_64546/3084144206.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(json.dumps(obj.dict(), indent=4))


In [29]:
jobs = client.fine_tuning.jobs.list()
print(jobs)

total=67 data=[CompletionJobOut(id='634c2e1a-6b03-4260-86f7-f9eb3fe9124f', auto_start=True, model='ministral-3b-latest', status='QUEUED', created_at=1750669586, modified_at=1750669586, training_files=['c69b7154-1577-43f1-b718-39aa4a9f9f70'], hyperparameters=CompletionTrainingParameters(training_steps=10, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=None, seq_len=32768, fim_ratio=None), validation_files=['8ce51668-728f-47cc-8f9c-0011ffae3ffa'], object='job', fine_tuned_model=None, suffix=None, integrations=[], trained_tokens=None, metadata=JobMetadataOut(expected_duration_seconds=None, cost=0.0, cost_currency=None, train_tokens_per_step=None, train_tokens=None, data_tokens=None, estimated_start_time=None), job_type='completion', repositories=[]), CompletionJobOut(id='631137c7-0d3f-4572-8d1c-64804008c44e', auto_start=True, model='ministral-3b-latest', status='CANCELLED', created_at=1750344590, modified_at=1750344596, training_files=['c69b7154-1577-43f1-b718-39aa4a

In [30]:
retrieved_jobs = client.fine_tuning.jobs.get(job_id = created_jobs.id)
retrieved_jobs

CompletionDetailedJobOut(id='634c2e1a-6b03-4260-86f7-f9eb3fe9124f', auto_start=True, model='ministral-3b-latest', status='QUEUED', created_at=1750669586, modified_at=1750669586, training_files=['c69b7154-1577-43f1-b718-39aa4a9f9f70'], hyperparameters=CompletionTrainingParameters(training_steps=10, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=None, seq_len=32768, fim_ratio=None), validation_files=['8ce51668-728f-47cc-8f9c-0011ffae3ffa'], object='job', fine_tuned_model=None, suffix=None, integrations=[], trained_tokens=None, metadata=JobMetadataOut(expected_duration_seconds=None, cost=0.0, cost_currency=None, train_tokens_per_step=None, train_tokens=None, data_tokens=None, estimated_start_time=None), job_type='completion', repositories=[], events=[], checkpoints=[])

In [ ]:
import time

retrieved_job = client.fine_tuning.jobs.get(job_id = created_jobs.id)
while retrieved_job.status in ["RUNNING", "QUEUED"]:
    retrieved_job = client.fine_tuning.jobs.get(job_id = created_jobs.id)
    pprint(retrieved_job)
    print(f"Job is {retrieved_job.status}, waiting 10 seconds")
    time.sleep(10)

/var/folders/bt/j58s9sqj2p534q65czjjc7fh0000gn/T/ipykernel_64546/3084144206.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(json.dumps(obj.dict(), indent=4))


{
    "id": "634c2e1a-6b03-4260-86f7-f9eb3fe9124f",
    "auto_start": true,
    "model": "ministral-3b-latest",
    "status": "QUEUED",
    "created_at": 1750669586,
    "modified_at": 1750669586,
    "training_files": [
        "c69b7154-1577-43f1-b718-39aa4a9f9f70"
    ],
    "hyperparameters": {
        "training_steps": 10,
        "learning_rate": 0.0001,
        "weight_decay": 0.1,
        "warmup_fraction": 0.05,
        "epochs": null,
        "seq_len": 32768,
        "fim_ratio": null
    },
    "validation_files": [
        "8ce51668-728f-47cc-8f9c-0011ffae3ffa"
    ],
    "object": "job",
    "fine_tuned_model": null,
    "suffix": null,
    "integrations": [],
    "trained_tokens": null,
    "metadata": {
        "expected_duration_seconds": null,
        "cost": 0.0,
        "cost_currency": null,
        "train_tokens_per_step": null,
        "train_tokens": null,
        "data_tokens": null,
        "estimated_start_time": null
    },
    "job_type": "completion",
    

In [ ]:
# List jobs
jobs = client.fine_tuning.jobs.list()
pprint(jobs)

In [ ]:
# Retrieve a jobs
retrieved_jobs = client.fine_tuning.jobs.get(job_id = created_jobs.id)
pprint(retrieved_jobs)

## Use a fine-tuned model

In [ ]:
chat_response = client.chat.complete(
    model = retrieved_jobs.fine_tuned_model,
    messages = [{"role":'user', "content":'A phase II, open-label, multicentre study to evaluate the immunogenicity and safety of an adjuvanted prepandemic (H5N1) influenza vaccine in healthy Japanese adults. Promising clinical data and significant antigen-sparing have been demonstrated for a pandemic H5N1 influenza split-virion vaccine adjuvanted with AS03A, an α-tocopherol-containing oil-in-water emulsion-based Adjuvant System. Although studies using this formulation have been reported, there have been no data for Japanese populations. This study therefore aimed to assess the immunogenicity and tolerability of a prepandemic (H5N1) influenza vaccine adjuvanted with AS03A in Japanese adults. This open-label, single-group study was conducted at two centres in Japan in healthy Japanese males and females aged 20-64 years (n = 100). Subjects received two doses of vaccine, containing 3.75 μg haemagglutinin of the A/Indonesia/5/2005-like IBCDC-RG2 Clade 2.1 (H5N1) strain adjuvanted with AS03A, 21 days apart. The primary endpoint evaluated the humoral immune response in terms of H5N1 haemagglutination inhibition (HI) antibody titres against the vaccine strain (Clade 2.1) 21 days after the second dose. Ninety five percent confidence intervals for geometric mean titres, seroprotection, seroconversion and seropositivity rates were calculated. Secondary and exploratory endpoints included the assessment of the humoral response in terms of neutralising antibody titres, the response against additional H5N1 strains (Clade 1 and Clade 2.2), as well as the evaluation of safety and reactogenicity. Robust immune responses were elicited after two doses of the prepandemic influenza vaccine adjuvanted with AS03A. Overall, vaccine HI seroconversion rates and seroprotection rates were 91% 21 days after the second vaccination. This fulfilled all regulatory acceptance criteria for the vaccine-homologous HI antibody level. A substantial cross-reactive humoral immune response was also observed against the virus strains A/turkey/Turkey/1/2005 (Clade 2.2) and A/Vietnam/1194/2004 (Clade 1) after the second vaccine administration. A marked post-vaccination response in terms of neutralising antibody titres was demonstrated and persistence of the immune response was observed 6 months after the first dose. The vaccine was generally well tolerated and there were no serious adverse events reported. The H5N1 candidate vaccine adjuvanted with AS03A elicited a strong and persistent immune response against the vaccine strain A/Indonesia/5/2005 in Japanese adults. Vaccination with this formulation demonstrated a clinically acceptable reactogenicity profile and did not raise any safety concerns in this population. Clinicaltrials.gov NCT00742885. Output:'}]
)

In [ ]:
pprint(chat_response)

## Integration with Weights and Biases
We can also offer support for integration with Weights & Biases (W&B) to monitor and track various metrics and statistics associated with our fine-tuning jobs. To enable integration with W&B, you will need to create an account with W&B and add your W&B information in the “integrations” section in the job creation request:



In [ ]:
client.fine_tuning.jobs.create(
    model="ministral-3b-latest",
    training_files=[{"file_id": ultrachat_chunk_train.id, "weight": 1}],
    validation_files=[ultrachat_chunk_eval.id],
    hyperparameters={"training_steps": 10, "learning_rate": 0.0001},
    integrations=[
        {
            "project": "<value>",
            "api_key": "<value>",
        }
    ]
)